In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

import numpy as np
import matplotlib.pyplot as plt
import sys
from Bio import SeqIO

In [ ]:
filename = "100k_rows.fasta"
#aa1_to_index = {'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4, 'G': 5, 'H': 6,
#                    'I': 7, 'K': 8, 'L': 9, 'M': 10, 'N': 11, 'P': 12,
#                    'Q': 13, 'R': 14, 'S': 15, 'T': 16, 'V': 17, 'W': 18,
#                    'Y': 19, 'X':20, 'NOSEQ':21, '-': 22}

acid_dict = {}
#aa1_to_index = {'A':0, 'B':1}

#print(aa1_to_index['B'])
print("Reading in data...", end='')
sys.stdout.flush()
seqs = []
counter = 0
count = 0
max_seq_len = 0
print(SeqIO.parse(filename, "fasta"))

num_seq = 1000000

for record in SeqIO.parse(filename, "fasta"):

    # Stupid way to generate the dictionary
    for elem in str(record.seq).upper():
        if not (elem in acid_dict):
            temp_arr = np.zeros(26)
            temp_arr[len(acid_dict)] = 1
            acid_dict[elem] = temp_arr
    max_seq_len = max(max_seq_len, len(record.seq))
    seqs.append(np.array([acid_dict[aa] for aa in str(record.seq).upper()]))
    #seqs.append(np.array([aa for aa in str(record.seq).upper()]))
    if (count > num_seq - 2):
        break
    else:
        count += 1


In [ ]:
append_val = np.array([0 if i != 25 else 1 for i in range(26)]).astype('float')

acid_dict.update({'-': append_val})

In [ ]:
print(len(seqs))
print(len(seqs[0]))
print(len(seqs[0][0]))
for i in range(len(seqs)):
     seqs[i] = np.pad(seqs[i],(0,(max_seq_len-seqs[i].shape[0])),'constant', constant_values=(0,acid_dict['-']))
       

In [ ]:
class LSTM_model():
    def __init__(self, input_size, hidden_layer_size, nr_hidden_layers):
        this.input_size = input_size
        this.hidden_layer_size = hidden_layer_size
        this.nr_hidden_layers = nr_hidden_layers
        
        this.model = nn.lstm(input_size, hidden_layer_size, nr_hidden_layers)
        
        this.linear = nn.linear(hidden_layer_size, )

In [ ]:

#seqs = np.random.choice(full_seqs, 100000, False)

max_length = max([len(seq) for seq in seqs])

data = torch.empty((len(seqs), max_length))
for i, seq in enumerate(seqs):
    #print(seq)
    data[i, :] = torch.from_numpy(seq)
print("done")

print("Data shape: ", data.shape)

train_samples = int(len(data) * 0.8)
train_data = torch.utils.data.TensorDataset(data[:train_samples])
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                               shuffle=True)
test_data = torch.utils.data.TensorDataset(data[train_samples:])
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                              shuffle=False)